In [1]:
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize

csv_filename = 'product_details_laptop.csv'
csv_bought_together = 'items_bought_together_laptop.csv'
csv_customer_reviews = 'links_to_customer_reviews_laptop.csv'
txt_customer_reviews = 'links_to_customer_reviews_laptop.txt'

In [2]:
df = pd.read_json('laptop_product_output.jsonl')
df.head()

,name,product_summary,product_tech_spec,product_addl_info,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews
0,CHUWI HeroBook Plus 15.6 inch Windows 10 Lapto...,"[{'info': 'Brand', 'value': 'CHUWI'}, {'info':...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B08HZ4KWJW'}, {'in...",ChuwiDirect,https://www.amazon.com/gp/help/seller/at-a-gla...,None,None,/CHUWI-HeroBook-Plus-Computer-Ethernet/product...
1,"LG Gram Thin & Light Laptop - 17"" IPS WQXGA (2...","[{'info': 'Brand', 'value': 'LG'}, {'info': 'S...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B08SW1LTST'}, {'in...",None,None,None,None,None
2,"ASUS Laptop L210 Ultra Thin Laptop, 11.6” HD D...","[{'info': 'Brand', 'value': 'ASUS'}, {'info': ...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B081V6W99V'}, {'in...",None,None,None,None,/ASUS-Processor-NumberPad-Microsoft-L210MA-DB0...
3,"Acer Nitro 5 Gaming Laptop, 10th Gen Intel Cor...","[{'info': 'Brand', 'value': 'Acer'}, {'info': ...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B08H2H89K1'}, {'in...",None,None,"VicTsing Mouse Pad with Stitched Edge, Premium...",/VicTsing-Stitched-Premium-Textured-Non-Slip-1...,/Acer-i5-10300H-GeForce-Keyboard-AN515-55-59KS...
4,"HP 15 Laptop, AMD Ryzen 3 3250U Processor, 8 G...","[{'info': 'Brand', 'value': 'HP'}, {'info': 'S...","[{'info': 'Standing screen display size', 'val...","[{'info': 'ASIN', 'value': 'B08N38VPL7'}, {'in...",None,None,None,None,/HP-Processor-Micro-Edge-Long-Lasting-15-ef102...


In [44]:
df.name = df.name.str.replace('"','inch')

In [3]:
df = df.dropna(axis=0, how='all')

In [4]:
df.link_to_all_reviews = 'https://www.amazon.com/' + df.link_to_all_reviews

In [5]:
df.reset_index(drop=True, inplace = True)

In [6]:
# Extraction of all specifications in nested JSON-dicts to be used for building additional columns 
  
new_columns = []
for i in range(len(df)):
    if df.product_summary[i] is not None:
        product_sum = df.product_summary[i]
        x = len(product_sum)  
        for j in range(x):
            new_columns.append(product_sum[j]['info'])
    if df.product_tech_spec[i] is not None:
        product_tech = df.product_tech_spec[i]
        y = len(product_tech)  
        for j in range(y):
            new_columns.append(product_tech[j]['info'])
    if df.product_addl_info[i] is not None:
        product_add = df.product_addl_info[i]
        z = len(product_add)  
        for j in range(z):
            new_columns.append(product_add[j]['info'])

In [7]:
for i in range(len(new_columns)):
    if new_columns[i] is not None:
        new_columns[i] = new_columns[i].lower()
new_columns = set(new_columns) 

In [8]:
new_columns = list(new_columns)

In [16]:
try:
    new_columns.remove(None)
except:
    pass

In [17]:
for i in new_columns:
    df[i] = i
    df[i] = ''

In [18]:
new_columns.sort()

In [24]:
len(new_columns)

93

In [25]:
for i in range(len(df)):
    if df.product_summary[i] is not None:
        psc = []
        product_sum = df.product_summary[i]
        product_sum = pd.DataFrame(product_sum)
        product_sum = product_sum.transpose()
        psc_len = len(product_sum.columns)  # collecting number of columns in product_sum / items in df.product_summary
        for j in range(psc_len):            # getting item / column names
            psc.append(product_sum[j][0])
        for k in range(len(psc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if psc[k] is not None:
                psc[k] = psc[k].lower()
        product_sum.columns = product_sum.iloc[0] # Turns row 1 into column names
        product_sum = product_sum.drop('info') # Drops row 1
        product_sum.columns = psc
        for l in psc: 
            try:
                df[l][i] = product_sum[l]['value']
            except:
                continue
    if df.product_tech_spec[i] is not None:
        ptc = []
        product_tec = df.product_tech_spec[i]
        product_tec = pd.DataFrame(product_tec)
        product_tec = product_tec.transpose()
        ptc_len = len(product_tec.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(ptc_len):            # getting item / column names
            ptc.append(product_tec[j][0])
        for k in range(len(ptc)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if ptc[k] is not None:
                ptc[k] = ptc[k].lower()
        product_tec.columns = product_tec.iloc[0] # Turns row 1 into column names
        product_tec = product_tec.drop('info') # Drops row 1
        product_tec.columns = ptc
        for l in ptc:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_tec[l]['value']
            except:
                continue
    if df.product_addl_info[i] is not None:
        pac = []
        product_add = df.product_addl_info[i]
        product_add = pd.DataFrame(product_add)
        product_add = product_add.transpose()
        pac_len = len(product_add.columns)  # collecting number of columns in product_tec / items in df.product_tech_spec
        for j in range(pac_len):            # getting item / column names
            pac.append(product_add[j][0])
        for k in range(len(pac)):           # setting item / column names in lower cases for transfer of values into correct df columns
            if pac[k] is not None:
                pac[k] = pac[k].lower()
        product_add.columns = product_add.iloc[0] # Turns row 1 into column names
        product_add = product_add.drop('info') # Drops row 1
        product_add.columns = pac
        for l in pac:   ## What does this loop through??? I need the column name, but I have to stay within one row!!!
            try:
                df[l][i] = product_add[l]['value']
            except:
                continue

In [26]:
df.rename(columns = {'asin' : 'ASIN'}, inplace = True)

In [29]:
df['customer reviews'] = df['customer reviews'].str.split('{\ntext-decoration: none;\n} ').str[1]
# df['customer reviews'] = df['customer reviews'].str.split(' out of 5 stars').str[0]
# df['customer reviews'] = df['customer reviews'].str.split(' von 5 Sternen').str[0]

# df['ASIN'] = df.url.str.split('/dp/').str[-1]
# df['ASIN'] = df.ASIN.str.split('%2Fdp%2F').str[-1]
# df['ASIN'] = df.ASIN.str.split('/').str[0]
# df['ASIN'] = df.ASIN.str.split('%2F').str[0]

In [33]:
pd.options.display.max_colwidth = 500
df['customer reviews']

KeyError: 'customer reviews'

In [31]:
for i in range(len(df)):
    if df['customer reviews'][i] is not None:
        try:
            res = [re.findall(r'(\d+).(\d+) out of 5 stars', df['customer reviews'][i])[0] ]
            a = res[0][0]
            b = res[0][1]
            c = float(a+'.'+b)
            df['customer reviews'][i] = c
        except:
            pass
    else:
        df['customer reviews'][i] = 0
df.rename(columns = {'customer reviews' : 'rating'}, inplace = True)

In [34]:
df.rating

0       3.4
1       NaN
2       4.3
3       4.6
4       4.7
       ... 
3580    4.4
3581    4.4
3582    4.5
3583    4.6
3584    4.3
Name: rating, Length: 3585, dtype: object

In [35]:
df = df.drop(['product_summary', 'product_tech_spec', 'product_addl_info'], axis = 1)

In [36]:
for i in range(len(df)):
    if df.name[i] is None:
        df = df.drop([i], axis = 0)

In [37]:
# Creating dataframe with links to all customer reviews for saving in additional CSV-file

df_laptop_customer_reviews_link = df.link_to_all_reviews
df_laptop_customer_reviews_link = df_laptop_customer_reviews_link.dropna()
df_laptop_customer_reviews_link.reset_index(drop=True, inplace = True)

In [38]:
df_laptop_bought_together = df[['ASIN', 'name', 'freq_bought', 'freq_bought_link']]

In [47]:
df.to_csv(csv_filename)
#df.to_csv(csv_filename, sep=';')
df_laptop_customer_reviews_link.to_csv(csv_customer_reviews) # actually not necessary anymore since there's being a txt-file created
df_laptop_bought_together.to_csv(csv_bought_together)

In [40]:
f = open(txt_customer_reviews, 'w+')

for i in range(len(df_laptop_customer_reviews_link)):
    f.write(df_laptop_customer_reviews_link[i])
    f.write('\n')

f.close()

In [41]:
df2 = pd.read_csv(csv_filename)
df2

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,manufacturer part number,memory speed,processor count,...,battery description,part number,package dimensions,line size,included components,manufacturer recommended age,folding,wireless type,domestic shipping,color screen
0,0,"CHUWI HeroBook Plus 15.6 inch Windows 10 Laptop, 1080P Laptop Computer with Intel J4125 and 12GB RAM / 256GB SSD, Support RJ45 Gigabit Ethernet, BT5.1, Dual WiFi",ChuwiDirect,https://www.amazon.com/gp/help/seller/at-a-glance.html/ref=dp_merchant_link?ie=UTF8&seller=A33LF4MGG0Q0UV&isAmazonFulfilled=1,NaN,NaN,https://www.amazon.com//CHUWI-HeroBook-Plus-Computer-Ethernet/product-reviews/B08HZ4KWJW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews,NaN,2 GHz,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bluetooth, 802.11bgn, 802.11ac",NaN,NaN
1,1,"LG Gram Thin & Light Laptop - 17"" IPS WQXGA (2560 x 1600), 11th Gen Intel Core i7 1165G7 CPU, Intel Iris Xe Graphics, 16GB RAM, 1TB NVMe SSD, 19.5 Hour Battery, - 17Z90P-K.AAB8U1 - Black (2021)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,802.11ax,NaN,NaN
2,2,"ASUS Laptop L210 Ultra Thin Laptop, 11.6” HD Display, Intel Celeron N4020 Processor, 4GB RAM, 64GB Storage, NumberPad, Windows 10 Home in S Mode with One Year of Microsoft 365 Personal, L210MA-DB01",NaN,NaN,NaN,NaN,https://www.amazon.com//ASUS-Processor-NumberPad-Microsoft-L210MA-DB01/product-reviews/B081V6W99V/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews,NaN,2.8 GHz,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bluetooth, 802.11ac",NaN,NaN
3,3,"Acer Nitro 5 Gaming Laptop, 10th Gen Intel Core i5-10300H,NVIDIA GeForce GTX 1650 Ti, 15.6"" Full HD IPS 144Hz Display, 8GB DDR4,256GB NVMe SSD,WiFi 6, DTS X Ultra,Backlit Keyboard,AN515-55-59KS",NaN,NaN,"VicTsing Mouse Pad with Stitched Edge, Premium-Textured Mouse Mat, Non-Slip Rubber Base Mousepad for…",/VicTsing-Stitched-Premium-Textured-Non-Slip-10-2%C3%978-3%C3%970-08/dp/B075WW7V6W/ref=pd_bxgy_2/131-0207570-2629776?_encoding=UTF8&pd_rd_i=B075WW7V6W&pd_rd_r=893c1d12-7777-49b0-98bb-9e1986776c22&pd_rd_w=7umrO&pd_rd_wg=qrUoG&pf_rd_p=f325d01c-4658-4593-be83-3e12ca663f0e&pf_rd_r=490AC3WY0QBY0FNXM175&psc=1&refRID=490AC3WY0QBY0FNXM175,https://www.amazon.com//Acer-i5-10300H-GeForce-Keyboard-AN515-55-59KS/product-reviews/B08H2H89K1/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews,NaN,4.5 GHz,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,802.11ax,NaN,NaN
4,4,"HP 15 Laptop, AMD Ryzen 3 3250U Processor, 8 GB RAM, 256 GB SSD Storage, 15.6-inch HD Micro-Edge Display, Windows 10 Home, Long-Lasting Battery, HP Fast Charge, 720p Webcam (15-ef1021nr, 2020)",NaN,NaN,NaN,NaN,https://www.amazon.com//HP-Processor-Micro-Edge-Long-Lasting-15-ef1021nr/product-reviews/B08N38VPL7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bluetooth, 802.11ax",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,3580,100Pcs Band Stickers Pack Rock Roll Stickers Decals Laptop Cars Guitar Bumper Punk Classic Vinyl Waterproof Graffiti Music Sticker,Amdi Rouu,https://www.amazon.com/gp/help/seller/at-a-glance.html/ref=dp_merchant_link?ie=UTF8&seller=A17FP619EZACVL&isAmazonFulfilled=1,"Rock Band Music Vinyl Stickers Pack 50pcs, Black and White Stickers Decals Laptop Cars Guitar Bumper…",https://www.amazon.com/Stickers-Decals-Classic-Waterproof-Graffiti/dp/B083YY2J63/ref=pd_bxgy_2/145-9855460-2861356?_encoding=UTF8&pd_rd_i=B083YY2J63&pd_rd_r=ca5f59cc-06bb-42fc-a3d6-bcafc8a2b42e&pd_rd_w=6m9BA&pd_rd_wg=HJ6zi&pf_rd_p=f325d01c-4658-4593-be83-3e12ca663f0e&pf_rd_r=4CV4NQNA71QJH9N5WZ08&psc=1&refRID=4CV4NQNA71QJH9N5WZ08,NaN,NaN,NaN,NaN,...,NaN,NaN,5.63 x 5.24 x 0.55 inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3578,3581,"Microsoft Surface Go (Intel Pentium Gold, 4GB RAM, 64GB)",NaN,NaN,Microsoft Surface Go Type Cover (Black),/Microsoft-Surface-Type-Cover-Black/dp/B07FDHK5NW/ref=pd_bxgy_2/134-0598617-7085429?_e

In [101]:
df2[df2.rating == 0]

,Unnamed: 0,name,seller,seller_link,freq_bought,freq_bought_link,link_to_all_reviews,brand name,color,rating,...,color name,ASIN,manufacturer part number,connectivity technologies,assembled width,special features,domestic shipping,os,assembled length,language


In [43]:
df2.columns

Index(['Unnamed: 0', 'name', 'seller', 'seller_link', 'freq_bought',
       'freq_bought_link', 'link_to_all_reviews', 'manufacturer part number',
       'memory speed', 'processor count', 'voltage', 'card description',
       'screen resolution', 'type of bulb', 'sheet size',
       'average battery life (in hours)', 'colour', 'department', 'language',
       'item thickness', 'style', 'audio jack', 'series', 'brand',
       'brand name', 'additional features', 'item package quantity',
       'finish types', 'material type', 'chipset brand', 'best sellers rank',
       'os', 'batteries required?', 'computer memory type',
       'item model number', 'color', 'operating system',
       'total recycled content percentage', 'weight', 'item weight',
       'warranty description', 'closure', 'import designation', 'ASIN',
       'rating', 'mfg recommended age', 'graphics card ram size',
       'hard drive interface', 'batteries required', 'number of usb 2.0 ports',
       'batteries', 'hardw